## Weight Purning

"Magnitude-based weight pruning **gradually zeroes out model weights during the training process to achieve model sparsity.**Sparse models are easier to compress, and we can skip the zeroes during inference for latency improvements."<br>
[Link to Official Page](https://www.tensorflow.org/model_optimization/guide/pruning)

#### <font color='red'> Can't Use TF Hub models because models are wrapped into KerasLayer Object which is not incompatible for optimization API at this moment</font>

In [1]:
import numpy as np
import tensorflow as tf 
import tensorflow_datasets as tfds

### Cats and Dogs Dataset

In [2]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

def format_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return  image, label

BATCH_SIZE = 32
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

train_batches = (raw_train.shuffle(num_examples // 4)
                 .map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

validation_batches = (raw_validation
                      .map(format_image)
                      .batch(BATCH_SIZE)
                      .prefetch(tf.data.AUTOTUNE))

test_batches = raw_test.map(format_image).batch(BATCH_SIZE)


### Model: Keras Transfer Learning

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [4]:
base_model = ResNet50(input_shape=(224,224,3),
                      include_top=False,
                      weights='imagenet',pooling='max')

# Customize the output layers
flatten_all = tf.keras.layers.Flatten()(base_model.output)
Dense_1 = tf.keras.layers.Dense(units=512,activation='relu')(flatten_all)
prediction_layer = tf.keras.layers.Dense(units=2, activation='softmax')(Dense_1)

# Concatenate the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

EPOCHS = 6
hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)

Epoch 1/6
582/582 [==============================] - 131s 213ms/step - loss: 0.4868 - accuracy: 0.9204 - val_loss: 0.7250 - val_accuracy: 0.4845
Epoch 2/6
582/582 [==============================] - 124s 212ms/step - loss: 0.1188 - accuracy: 0.9528 - val_loss: 0.3261 - val_accuracy: 0.8508
Epoch 3/6
582/582 [==============================] - 124s 212ms/step - loss: 0.1084 - accuracy: 0.9582 - val_loss: 0.1666 - val_accuracy: 0.9261
Epoch 4/6
582/582 [==============================] - 123s 211ms/step - loss: 0.0966 - accuracy: 0.9637 - val_loss: 0.1556 - val_accuracy: 0.9424
Epoch 5/6
582/582 [==============================] - 124s 212ms/step - loss: 0.0918 - accuracy: 0.9657 - val_loss: 0.4711 - val_accuracy: 0.8551
Epoch 6/6
582/582 [==============================] - 124s 212ms/step - loss: 0.0865 - accuracy: 0.9675 - val_loss: 0.3011 - val_accuracy: 0.8908


In [5]:
baseline_model_accuracy = model.evaluate(test_batches)

73/73 [==============================] - 4s 61ms/step - loss: 0.3047 - accuracy: 0.8934 0s - loss: 0.3041 - accuracy


In [6]:
tf.saved_model.save(model,'model_dir/ResNet/')

INFO:tensorflow:Assets written to: model_dir/ResNet/assets


INFO:tensorflow:Assets written to: model_dir/ResNet/assets


#### Pruning the weights

In [7]:
import tensorflow_model_optimization as tfmot

In [9]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
BATCH_SIZE = 32
EPOCHS = 2

num_images = num_examples
end_step = np.ceil(num_images / BATCH_SIZE).astype(np.int32) * EPOCHS

print("Number of Images:",num_images)
print("End Steps:", end_step)

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# model_for_pruning.summary()

Number of Images: 23262
End Steps: 1454


In [10]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='log_dir_purning/'),
]

model_for_pruning.fit(train_batches,
                      epochs=EPOCHS, 
                      validation_data=validation_batches,
                      callbacks=callbacks)

C:\Users\user\anaconda3\envs\tensorflow2\lib\site-packages\keras\layers\wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config = {'layer': generic_utils.serialize_keras_object(self.layer)}


Epoch 1/2
  6/582 [..............................] - ETA: 2:16 - loss: 0.1062 - accuracy: 0.9531WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1363s vs `on_train_batch_end` time: 0.1903s). Check your callbacks.


582/582 [==============================] - 167s 248ms/step - loss: 0.1008 - accuracy: 0.9610 - val_loss: 0.1152 - val_accuracy: 0.9514
Epoch 2/2
582/582 [==============================] - 145s 248ms/step - loss: 0.0618 - accuracy: 0.9759 - val_loss: 0.1181 - val_accuracy: 0.9557


In [11]:
model_for_pruning.evaluate(test_batches)

73/73 [==============================] - 5s 63ms/step - loss: 0.1086 - accuracy: 0.9536


[0.10859815776348114, 0.9535683393478394]

In [12]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

tf.saved_model.save(model_for_export,'model_dir/Pruning/')

INFO:tensorflow:Assets written to: model_dir/Pruning/assets


INFO:tensorflow:Assets written to: model_dir/Pruning/assets
